In [1]:
import pybullet as p
import time
import pybullet_data
import eigen as e
import rbdyn as rbd
import sva as s
import numpy as np
from rbdyn.parsers import *

DT = 0.001 #s
MOVEMENT_TIME = 10000 #ms
URDF = "./resource/kuka_iiwa.urdf"

def get_rot(T):
    R = e.Matrix3d()
    for row in range(R.rows()):
        for col in range(R.cols()):
            R.coeff(row, col, T.coeff(row, col))
    return R

def get_pos(T):
    pos = e.Vector3d()
    for i in range(pos.rows()):
        pos[i] = T.coeff(i,3)
    return pos

def get_col(R, c=0):
    assert c <= R.cols(), "column index out of bounds"
    assert R.rows() == R.cols(), "square matrix expected"
    assert R.rows() == 3, "3x3 matrix expected"
    v = e.Vector3d()
    for i in range(R.rows()):
        v[i] = R.coeff(i,c)
    return v

def get_orientation_error(R, R_d):
    '''Calculates orientation error
    @param R Current orientation in Mat3d
    @param R_d Desired orientation in Mat3d
    @return del_phi "R_d-R" error
    '''
    c1 = get_col(R,0)
    c2 = get_col(R,1)
    c3 = get_col(R,2)
    d1 = get_col(R_d,0)
    d2 = get_col(R_d,1)
    d3 = get_col(R_d,2)
    
    return 1.0/2.0*(c1.cross(d1)+c2.cross(d2)+c3.cross(d3))
    
    
def body_id_from_name(name, bodies):
    '''Gets the body Id from the body name
    @param name The name of the body
    @param bodies The set of bodies provided by the multibody data structure
    @return Id of the body, -1 if not found
    '''
    for bi, b in enumerate(bodies):
        if (b.name().decode("utf-8") == name):
            return bi
    return -1

def sva_to_affine(s):
    '''Converts a spatial transform matrix to a homogeneous transform matrix
    @param s Spatial transform
    @return Homogeneous transform matrix
    '''
    m4d = e.Matrix4d.Identity()
    R = s.rotation().transpose()
    pos = s.translation();
    
    for row in range(R.rows()):
        for col in range(R.cols()):
            m4d.coeff(row, col, R.coeff(row, col))
    for row in range(pos.rows()):
        m4d.coeff(row, 3, pos[row]);
        
    return m4d

def inv_spd_sr(mat, k=1e-4):
    '''Converts an inverse into an SVD problem with singularity robust inverse
    @param mat Matrix to be inversed
    @param tolerance Tolerance to the minimum singular value that can go through reciprocal operation
    @return Homogeneous transform matrix
    '''
    mat_np = np.array(mat)
    reg = k * np.identity(np.size(mat_np,0))
    
    return e.MatrixXd(mat_np.transpose().dot(np.linalg.inv(mat_np.dot(mat_np.transpose()) + reg)))
    
# import robot as global variable
r = from_urdf_file(URDF)
print("Imported " + r.name.decode("utf-8"))
# set gravity direction (this is the acceleration at base joint for RNEA)
r.mbc.gravity = e.Vector3d(0,0,9.81)
r.mbc.zero(r.mb)

# setup physics
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #used by loadURDF
p.setGravity(0,0,-9.81)
p.setRealTimeSimulation(0)
p.setTimeStep(DT)

# import robot
planeId = p.loadURDF("plane.urdf")
startPos = [0,0,0]
startOrientation = p.getQuaternionFromEuler([0,0,0])
loadFlag = p.URDF_USE_INERTIA_FROM_FILE | p.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS
robotId = p.loadURDF(URDF,startPos, startOrientation, flags=loadFlag)

# parse number of joints except fixed joint
print("Simulation can only handle revolute and fixed joints")
jointsId = []
for i in range(p.getNumJoints(robotId)):
    if(p.getJointInfo(robotId, i)[2] == p.JOINT_REVOLUTE):
        jointsId.append(i)
jointsId = np.array(jointsId)
nDof = len(jointsId)

# gains
Kp = e.MatrixXd(np.diag([10000,10000,10000,40000,40000,40000]))
Kd = e.MatrixXd(np.diag([200,200,200,400,400,400]))

# selection matrix to swap linear and angular components
S = e.MatrixXd(6,6)
S.coeff(0,3,1)
S.coeff(1,4,1)
S.coeff(2,5,1) # linear
S.coeff(3,0,1)
S.coeff(4,1,1)
S.coeff(5,2,1) # angular

# task definitions
bodyName1 = "lbr_iiwa_link_7"
S1 = S
# bodyName2 = "lbr_iiwa_link_3"
# S2 = e.MatrixXd(3,6)
# S2.coeff(0,3,1)
# S2.coeff(1,4,1)
# S2.coeff(2,5,1)

# setup configuration
p.setJointMotorControlArray(robotId, jointsId, p.VELOCITY_CONTROL, forces=[0] * nDof)
q_home = np.zeros(nDof)
for i in jointsId:
    p.resetJointState(robotId, i, targetValue=q_home[i])
    p.enableJointForceTorqueSensor(robotId, i)
q_home = e.VectorXd(q_home)

pybullet build time: Mar 26 2022 03:00:52


Imported lbr_iiwa
Simulation can only handle revolute and fixed joints


In [ ]:
def setQInMbc(q):
    for i in range(0, nDof):
        r.mbc.q[i+1][0] = q[i]

def setDqInMbc(dq):
    for i in range(0, nDof):
        r.mbc.alpha[i+1][0] = dq[i]
        
# gains
Kp_joint = e.MatrixXd(np.diag([8000]*nDof))
Kd_joint = e.MatrixXd(np.diag([180]*nDof))
Ko_joint = e.MatrixXd(np.diag([100]*nDof))
    
# observer
residual = e.VectorXd([0] * nDof)
residual_int = e.VectorXd([0] * nDof)
    
# simulate torque control
for i in range(1000000):
    joint_states = p.getJointStates(robotId, jointsId)
    
    # update q and dq states from simulation
    qBullet = [joint_states[i][0] for i in jointsId]
    dqBullet = [joint_states[i][1] for i in jointsId]

    # update states into mbc
    setQInMbc(qBullet)
    setDqInMbc(dqBullet)
    
    # updates states into "eigen" representation
    q = e.VectorXd(qBullet)
    dq = e.VectorXd(dqBullet)

    ############################### forward kinematics to update model ###############################
    rbd.forwardKinematics(r.mb, r.mbc)
    rbd.forwardVelocity(r.mb, r.mbc)
    
    # pose
    ofsRot = e.Matrix3d.Identity()
    ofsPos = e.Vector3d(0,0,0)
    T1 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * r.mbc.bodyPosW[body_id_from_name(bodyName1, r.mb.bodies())])

    # jacobian, dJacobian [angular; linear]
    jac = rbd.Jacobian(r.mb, bodyName1.encode('utf-8'), ofsPos)
    tempJ1 = e.MatrixXd(6, r.mb.nrDof());
    tempdJ1 = e.MatrixXd(6, r.mb.nrDof());
    jac.fullJacobian(r.mb, jac.jacobian(r.mb, r.mbc), tempJ1);
    jac.fullJacobian(r.mb, jac.jacobianDot(r.mb, r.mbc), tempdJ1);
    J1 = S*tempJ1
    dJ1 = S*tempdJ1
    
    ############################### forward dynamics to update model ###############################
    fd = rbd.ForwardDynamics(r.mb)
    
    # mass matrix & nonlinear effects 
    fd.computeH(r.mb, r.mbc)
    M = fd.H()
    fd.computeC(r.mb, r.mbc)
    h = fd.C()
    
    # coriolis matrix
    cor = rbd.Coriolis(r.mb)
    C = cor.coriolis(r.mb, r.mbc)
    
    # gravity
    setDqInMbc([0] * nDof)
    rbd.forwardKinematics(r.mb, r.mbc)
    fd = rbd.ForwardDynamics(r.mb)
    fd.computeC(r.mb, r.mbc)
    g = fd.C()
    setDqInMbc(dqBullet)
    rbd.forwardKinematics(r.mb, r.mbc)
    rbd.forwardVelocity(r.mb, r.mbc)
    
    # apply force
    p.applyExternalForce(robotId, jointsId[-1], (0,0,0), (0,0,0), p.LINK_FRAME)

    # control
    err = e.VectorXd([0,np.pi/4,0,np.pi/4,0,np.pi/4,0])-q
    derr = -dq
    tau = e.VectorXd(M*(Kp_joint*err+Kd_joint*derr)) + h

    # momentum observer
    beta = g - C.transpose()*dq
    residual_int += (residual+tau+C.transpose()*dq-g)*DT
    residual = Ko_joint*(M*dq-residual_int)
    Jc = e.MatrixXd(J1)
    JcT = Jc.transpose()
    JcT_inv = e.MatrixXd(np.linalg.pinv(JcT))
    f = JcT_inv*residual
    print("{}\n".format(f.transpose()))
    
    # command system:
    for i in jointsId:
        p.setJointMotorControl2(robotId, i, p.TORQUE_CONTROL, force=tau[i])
        
    # simulate
    p.stepSimulation()
    time.sleep(DT)
p.disconnect()

    -3452.89  3.96388e+15 -1.86589e+15  2.76293e+15      1244.34     -579.449

-701.611  2089.12   164387  1195.88  130.676 -10.0653

-469.908  502.812  39775.7  288.813  86.7031 -6.77497

-285.462  165.289  13308.7  95.4311  52.2522 -4.16976

-141.102  53.5868  4544.68  31.5184  25.5411 -2.14146

 -30.7973   9.13443    1036.4   6.33527    5.1055 -0.598153

  50.633 -9.94961 -496.301 -4.16104 -10.2567 0.539016

 107.731 -18.0722 -1178.56 -8.28708 -21.5317  1.33839

 144.536 -21.0519  -1463.1 -9.43164 -29.5283  1.85958

 164.664 -21.5038 -1552.24 -9.13656 -34.9101  2.15475

 171.347 -20.6947 -1543.15 -8.15945 -38.2216  2.26945

 167.459 -19.2718 -1484.84 -6.89261  -39.909  2.24328

 155.528 -17.5785 -1402.85  -5.5456 -40.3377  2.11035

 137.745 -15.8018 -1310.75 -4.23125 -39.8065  1.89974

 115.979 -14.0441 -1215.79 -3.00897 -38.5585   1.6359

 91.7978 -12.3617 -1121.84 -1.90764 -36.7913  1.33908

  66.4872  -10.7839  -1030.92 -0.938348   -34.664   1.02574

  41.0772  -9.32489  -944.009